![Asesinos Header](data/Asesinos_detective_agency.png)

El enunciado nos da las sigueintes pistas:
- Sabemos que fue un asesinato 
- Ocurrio el 15 de enero de 2018
- En fue en la ciudad SQL City

In [1]:
import pandas as pd
import sqlite3

In [2]:
#Me conecto a la base de datos Chinook.db que tenemos la ruta 
connection=sqlite3.connect("./data/sql-murder-mystery.db")
#me creo un cursor 
cursor_TC_Murder=connection.cursor()

In [3]:
# Con esta función leemos los datos y lo pasamos a un DataFrame de Pandas como nos dice Jaime en los videos 
def sql_query(query):

    # Ejecuta la query
    cursor_TC_Murder.execute(query) # Recuerda que sólo funcionará si has llamado cursor_bootcamp
                                    # a tu cursor, si no, cambia el nombre en todo el código de la función

    # Almacena los datos de la query(fetchall() devuelve todas las filas de la consulta en una lista de tuplas)
    ans = cursor_TC_Murder.fetchall()

    # Obtenemos los nombres de las columnas de la tabla
    names = [description[0] for description in cursor_TC_Murder.description]

    return pd.DataFrame(ans,columns=names)

Con esta información realizamos una consulta en la tabla crime_scene_report filtrando por fecha y ciudad. Allí encontramos la descripción del crimen y descubrimos que había dos testigos:  

In [4]:
query= '''SELECT *
FROM crime_scene_report
WHERE date="20180115" AND city="SQL City"
'''
sql_query(query)

,date,type,description,city
0,20180115,assault,"Hamilton: Lee, do you yield? Burr: You shot hi...",SQL City
1,20180115,assault,Report Not Found,SQL City
2,20180115,murder,Security footage shows that there were 2 witne...,SQL City


**2. Informacion de los testigos:**

Como se ve en la imagen encontramos en la columna de type en la tabla crime_scene_report vemos la descripción del asesinato y que había dos testigos el primero vivía en la casa de Northwestern Dr y el segundo testigo se llama Anabel y vive en Franklin Ave. Decidimos con esa información hacer dos búsquedas primero la de anabel que tenemos tambien su dirección. Y después con el segundo testigo 

**3. Declaraciones de los testigos**

1) Primera búsqueda de opción 1 testigo: calle (Franklin Ave) y nombre (Annabel) 

In [5]:
query= '''SELECT *
FROM person
WHERE address_street_name="Franklin Ave" AND name LIKE "%Annabel%" '''

sql_query(query)

,id,name,license_id,address_number,address_street_name,ssn
0,16371,Annabel Miller,490173,103,Franklin Ave,318771143


De aquí sacamos su id , que es 16371. 

2) Segunda búsqueda de opción 2 testigo: Northwestern Dr 

In [6]:
query='''SELECT id, name, address_number, address_street_name
FROM person
WHERE address_street_name="Northwestern Dr" 
ORDER BY address_number DESC LIMIT 1'''
sql_query(query)

,id,name,address_number,address_street_name
0,14887,Morty Schapiro,4919,Northwestern Dr


Aquí localizamos su ID que esa el 14887. 

**4. Declaraciones de los testigos**

Ahora con los dos ID obtenidos hacemos una búsqueda de los dos ID en la tabla interview para ver que dijeron a la policía su atestado. 

In [7]:
query='''SELECT  person_id, transcript
FROM interview
WHERE person_id IN (14887, 16371)'''

sql_query(query)

,person_id,transcript
0,14887,I heard a gunshot and then saw a man run out. ...
1,16371,"I saw the murder happen, and I recognized the ..."


De aquí sacamos que Monty escucho un disparo y vio a un hombre salir corriendo llevaba la bolsa del gimnasio GET FIT NOW y el numero de membresía empezaba por 48Z  y esas bolsas solo la tienen miembros gold, y también vio que se subió a un coche con la matricula que contiene los caracteres H42W. Y de Ana sacamos la información de que vio el asesinato y reconoció al asesino de su gimnasio cuando estaba entrenando el 9 de enero. 

**5. Seguimiento de membresia en el gym y matricula**

Gracias a la información proporcionada por los testigos, podemos cotejar a los miembros del gimnasio con estatus oro con la información de la matrícula, vinculándolos a través del ID de las personas. De este modo, encontramos que toda la información apunta a una sola persona…

In [11]:
query='''
SELECT a.name,a.person_id,a.membership_status,a.id,b.check_in_date,d.plate_number
FROM get_fit_now_member AS a
LEFT JOIN get_fit_now_check_in AS b ON a.id == b.membership_id
LEFT JOIN person AS c ON a.person_id == c.id
LEFT JOIN drivers_license AS d ON c.license_id == d.id
WHERE a.membership_status = "gold" AND a.id like "48Z%" AND b.check_in_date = "20180109" AND d.plate_number LIKE "%H42W%"
'''

sql_query(query)

,name,person_id,membership_status,id,check_in_date,plate_number
0,Jeremy Bowers,67318,gold,48Z55,20180109,0H42W2


Congrats, you found the murderer! But wait, there's more... If you think you're up for a challenge, try querying the interview transcript of the murderer to find the real villain behind this crime. If you feel especially confident in your SQL skills, try to complete this final step with no more than 2 queries. Use this same INSERT statement with your new suspect to check your answer.

**6. El testimonio del señor Bowers**

Pero espera, hay más en este crimen de lo que pensábamos. Parece que el señor Bowers actuaba en nombre de otra persona. Tenemos que investigar más a fondo para descubrir quién fue el cerebro criminal. Con su ID podemos localizar la transcripción de sus entrevistas.

In [22]:
query='''
SELECT person_id,transcript
FROM interview
WHERE person_id = 67318
'''

sql_query(query)

,person_id,transcript
0,67318,I was hired by a woman with a lot of money. I ...


El señor Bowers iba a caer solo y no tardó en delatar a la persona que lo contrató. Proporcionó información valiosa como el hecho de que se trata de una mujer, la estatura, el color de cabello, el tipo y modelo de su coche e incluso que ella asistió a cierto concierto tres veces en el año 2017.

Tomando la información proporcionada por el muy generoso señor Bowers, podemos usar los datos sobre la marca y el modelo del coche, junto con la estatura, el color de cabello y el género, para cotejar una identificación con la de alguien que asistió al concierto en cuestión tres veces en 2017.

In [23]:
query= '''
SELECT a.height,a.hair_color,a.gender,a.car_make,a.car_model,b.name,b.id,c.event_name,c.date
FROM drivers_license AS a
LEFT JOIN person AS b ON a.id == b.license_id
LEFT JOIN facebook_event_checkin AS c ON b.id == c.person_id
WHERE a.height between 65 and 67 and a.hair_color = "red" and a.gender = "female" and a.car_make = "Tesla" and a.car_model like "%S%" and c.event_name = "SQL Symphony Concert" and c.date like "2017%"
'''              
sql_query(query)


              

,height,hair_color,gender,car_make,car_model,name,id,event_name,date
0,66,red,female,Tesla,Model S,Miranda Priestly,99716,SQL Symphony Concert,20171206
1,66,red,female,Tesla,Model S,Miranda Priestly,99716,SQL Symphony Concert,20171212
2,66,red,female,Tesla,Model S,Miranda Priestly,99716,SQL Symphony Concert,20171229


Congrats, you found the brains behind the murder! Everyone in SQL City hails you as the greatest SQL detective of all time. Time to break out the champagne!

¡Y ahí lo tenemos! Toda la información nos da un resultado concluyente. ¡La perpetradora es Miranda Priestly! Es hora de poner a estos criminales entre rejas. ¡Otro caso resuelto por la agencia de detectives Asesinos!